In [11]:
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

#Definimos la función de la distribución de Fermi-Dirac
def fermi_dirac(E, EF, kT):
    return 1 / (1 + np.exp((E - EF) / kT))

#Función para encontrar el cero usando el método de bisección
def bisection_method(func, a, b, tol=1e-6, max_iter=1000):
    start_time = time.perf_counter()  # Tomamos el tiempo inicial

    #Inicializamos variables
    x_prev = None
    error = None
    num_iterations = 0

    #Loop de iteraciones
    while num_iterations < max_iter:
        num_iterations += 1

        #Calculamos el punto medio
        x = (a + b) / 2
        E = x  #Valor de E

        # Evaluamos la función en el punto medio y en los extremos del intervalo
        fa = func(a)
        fb = func(b)
        fx = func(x)

        # Calculamos el error
        if x_prev is not None:
            error = max(abs(fx), abs(x - x_prev))

        #Se comprueba si la raíz está en el intervalo [a, x] o [x, b]
        if fa * fx < 0:
            b = x
        else:
            a = x

        #Se comprueba el criterio de convergencia
        if error is not None and error < tol:
            break

        x_prev = x  #Se actualiza x_prev para la siguiente iteración

    end_time = time.perf_counter()                   #Tomamos el tiempo final
    execution_time = (end_time - start_time) * 1000  #Calculamos el tiempo de ejecución en milisegundos

    return x, error, num_iterations, execution_time

#Parámetros para la función de la distribución de Fermi-Dirac
EF = 3.0       #Energía de activación
kT = 4.14e-21  #Energía térmica

#Definimos los valores iniciales para el intervalo de bisección
a = EF - 10
b = EF + 10

#Se aplica el método de bisección
zero, error, iterations, exec_time = bisection_method(lambda E: fermi_dirac(E, EF, kT), a, b)

#Resultados finales
print("\nResultado de la bisección:")
print("Intervalo inicial [a, b]:", a, ",", b)
print("Raíz encontrada:", zero)
print("Error:", error)
print("Número de iteraciones:", iterations)
print("Tiempo de ejecución:", exec_time, "milisegundos")



Resultado de la bisección:
Intervalo inicial [a, b]: -7.0 , 13.0
Raíz encontrada: 12.999999403953552
Error: 5.960464477539062e-07
Número de iteraciones: 25
Tiempo de ejecución: 0.26000000070780516 milisegundos


In [22]:
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

#Definimos la función de la distribución de Fermi-Dirac
def fermi_dirac(E, EF, kT):
    return 1 / (1 + np.exp((E - EF) / kT))

#Función para encontrar el cero usando el método de la falsa posición
def false_position(func, a, b, tol=1e-6, max_iter=1000):
    start_time = time.perf_counter()  #Tomamos el tiempo inicial

    #Inicializamos variables
    x_prev = None
    error = None
    num_iterations = 0

    # Bucle de iteraciones
    while num_iterations < max_iter:
        num_iterations += 1

        #Calculamos la nueva aproximación
        fa = func(a)
        fb = func(b)
        x = b - (fb * (b - a)) / (fb - fa)
        E = x  #Valor de E

        #Evaluamos la función en la nueva aproximación
        fx = func(x)

        #Verificamos si hemos encontrado la raíz o si necesitamos ajustar el intervalo
        if np.sign(fx) == np.sign(fa):
            a = x
        else:
            b = x

        #Calculamos el error
        if x_prev is not None:
            error = abs(x - x_prev)

        #Se comprueba el criterio de convergencia
        if error is not None and error < tol:
            break

        x_prev = x  #Se actualiza x_prev para la siguiente iteración

    end_time = time.perf_counter()                   #Tomamos el tiempo final
    execution_time = (end_time - start_time) * 1000  #Calculamos el tiempo de ejecución en milisegundos

    return x, error, num_iterations, execution_time

#Parámetros para la función de la distribución de Fermi-Dirac
EF = 3.0  # Energía de activación
kT = 4.14e-21  # Energía térmica

#Se define el intervalo inicial
a = EF - 10  # Comenzamos desde E = 3
b = EF + 10  # Terminamos en E = 13

#Se aplica el método de la falsa posición
zero, error, iterations, exec_time = false_position(lambda E: fermi_dirac(E, EF, kT), a, b)

# Imprimimos los resultados finales
print("\nResultado del Método de la falsa posición:")
print("Intervalo inicial [a, b]:", a, ",", b)
print("Raíz encontrada:", zero)
print("Error:", error)
print("Número de iteraciones:", iterations)
print("Tiempo de ejecución:", exec_time, "milisegundos")



Resultado del Método de la falsa posición:
Intervalo inicial [a, b]: -7.0 , 13.0
Raíz encontrada: 13.0
Error: 0.0
Número de iteraciones: 2
Tiempo de ejecución: 0.07250000271596946 milisegundos


In [23]:
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

#Definimos la función de la distribución de Fermi-Dirac
def fermi_dirac(E, EF, kT):
    return 1 / (1 + np.exp((E - EF) / kT))

#Función para encontrar el cero usando el método de Muller-Krumbholz (MK6)
def muller_krumbholz_6(func, x0, x1, x2, tol=1e-10, max_iter=1000):
    start_time = time.perf_counter()  # Tomamos el tiempo inicial
    
    #Inicializamos variables
    x_prev = None
    error = None
    num_iterations = 0
    
    #Bucle de iteraciones
    while num_iterations < max_iter:
        num_iterations += 1
        
        #Calculamos los coeficientes del polinomio cuadrático
        h1 = x1 - x0
        h2 = x2 - x1
        delta1 = (func(x1) - func(x0)) / h1
        delta2 = (func(x2) - func(x1)) / h2
        d = (delta2 - delta1) / (h2 + h1)
        b = delta2 + h2 * d
        D = np.sqrt(b ** 2 - 4 * func(x2) * d)
        
        #Elegimos el punto más cercano a x2 como nueva aproximación
        if abs(b - D) < abs(b + D):
            E = b + D
        else:
            E = b - D
        
        #Actualizamos las aproximaciones
        h = -2 * func(x2) / E
        p = x2 + h
        x_prev = x2
        x2 = p
        
        #Calculamos el error
        if x_prev is not None:
            error = abs(x2 - x_prev)
        
        #Comprobamos el criterio de convergencia
        if error is not None and error < tol:
            break
        
    end_time = time.perf_counter()                   #Tomamos el tiempo final
    execution_time = (end_time - start_time) * 1000  #Calculamos el tiempo de ejecución en milisegundos
    
    return x2, error, num_iterations, execution_time

#Parámetros para la función de la distribución de Fermi-Dirac
EF = 3.0  # Energía de activación
kT = 4.14e-21  # Energía térmica

#Definimos las aproximaciones iniciales
x0 = EF - 10
x1 = EF
x2 = EF + 10

#Aplicamos el método de Muller-Krumbholz (MK6)
zero, error, iterations, exec_time = muller_krumbholz_6(lambda E: fermi_dirac(E, EF, kT), x0, x1, x2)

#Imprimimos los resultados
print("Resultado del Método de Muller-Krumbholz (MK6):")
print("Intervalo inicial [x0, x1, x2]:", x0, ",", x1, ",", x2)
print("Raíz encontrada:", zero)
print("Error:", error)
print("Número de iteraciones:", iterations)
print("Tiempo de ejecución:", exec_time, "milisegundos")


Resultado del Método de Muller-Krumbholz (MK6):
Intervalo inicial [x0, x1, x2]: -7.0 , 3.0 , 13.0
Raíz encontrada: 13.0
Error: 0.0
Número de iteraciones: 1
Tiempo de ejecución: 0.11180000001331791 milisegundos


In [21]:
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

#Definimos la función de la distribución de Fermi-Dirac
def fermi_dirac(E, EF, kT):
    return 1 / (1 + np.exp((E - EF) / kT))

#Función para encontrar el cero usando el método de Muller-Krumbholz (MK7)
def muller_krumbholz_7(func, x0, x1, x2, tol=1e-10, max_iter=1000):
    start_time = time.perf_counter()  # Tomamos el tiempo inicial
    
    #Inicializamos variables
    x_prev = None
    error = None
    num_iterations = 0
    
    #Bucle de iteraciones
    while num_iterations < max_iter:
        num_iterations += 1
        
        #Calculamos los coeficientes del polinomio cúbico
        h1 = x1 - x0
        h2 = x2 - x1
        delta1 = (func(x1) - func(x0)) / h1
        delta2 = (func(x2) - func(x1)) / h2
        d = (delta2 - delta1) / (h2 + h1)
        b = delta2 + h2 * d
        D = np.sqrt(b ** 2 - 4 * func(x2) * d)
        
        #Elegimos el punto más cercano a x2 como nueva aproximación
        if abs(b - D) < abs(b + D):
            E = b + D
        else:
            E = b - D
        
        #Actualizamos las aproximaciones
        h = -2 * func(x2) / E
        p = x2 + h
        x_prev = x2
        x2 = p
        
        #Calculamos el error
        if x_prev is not None:
            error = abs(x2 - x_prev)
        
        #Comprobamos el criterio de convergencia
        if error is not None and error < tol:
            break
        
    end_time = time.perf_counter()                   #Tomamos el tiempo final
    execution_time = (end_time - start_time) * 1000  #Calculamos el tiempo de ejecución en milisegundos
    
    return x2, error, num_iterations, execution_time

#Parámetros para la función de la distribución de Fermi-Dirac
EF = 3.0       #Energía de activación
kT = 4.14e-21  #Energía térmica

#Definimos las aproximaciones iniciales
x0 = EF - 10
x1 = EF
x2 = EF + 10

#Aplicamos el método de Muller-Krumbholz (MK7)
zero, error, iterations, exec_time = muller_krumbholz_7(lambda E: fermi_dirac(E, EF, kT), x0, x1, x2)

#Resultados
print("Resultado del Método de Muller-Krumbholz (MK7):")
print("Intervalo inicial [x0, x1, x2]:", x0, ",", x1, ",", x2)
print("Raíz encontrada:", zero)
print("Error:", error)
print("Número de iteraciones:", iterations)
print("Tiempo de ejecución:", exec_time, "milisegundos")


Resultado del Método de Muller-Krumbholz (MK7):
Intervalo inicial [x0, x1, x2]: -7.0 , 3.0 , 13.0
Raíz encontrada: 13.0
Error: 0.0
Número de iteraciones: 1
Tiempo de ejecución: 0.06260000009206124 milisegundos
